In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-26 23:40:32.098551: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 23:40:32.125564: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 23:40:32.125614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 23:40:32.125645: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 23:40:32.133015: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-26 23:40:33.775189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:40:33.799224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 23:40:33.801078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [11]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
#IMG_SIZE = (224, 224)


In [12]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [13]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [14]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.
Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [15]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [16]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [17]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [18]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [19]:
import tensorflow.keras.applications as apps
base_model = apps.MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [20]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: Conv1, False
Layer: bn_Conv1, False
Layer: Conv1_relu, False
Layer: expanded_conv_depthwise, False
Layer: expanded_conv_depthwise_BN, False
Layer: expanded_conv_depthwise_relu, False
Layer: expanded_conv_project, False
Layer: expanded_conv_project_BN, False
Layer: block_1_expand, False
Layer: block_1_expand_BN, False
Layer: block_1_expand_relu, False
Layer: block_1_pad, False
Layer: block_1_depthwise, False
Layer: block_1_depthwise_BN, False
Layer: block_1_depthwise_relu, False
Layer: block_1_project, False
Layer: block_1_project_BN, False
Layer: block_2_expand, False
Layer: block_2_expand_BN, False
Layer: block_2_expand_relu, False
Layer: block_2_depthwise, False
Layer: block_2_depthwise_BN, False
Layer: block_2_depthwise_relu, False
Layer: block_2_project, False
Layer: block_2_project_BN, False
Layer: block_2_add, False
Layer: block_3_expand, False
Layer: block_3_expand_BN, False
Layer: block_3_expand_relu, False
Layer: block_3_pad, False
Layer: block_3_d

In [21]:
len(base_model.layers)

154

In [22]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [23]:
aug_layer = utils.return_data_aug_layer_for_eff_net(random_width=0.3, random_height=0.3, random_zoom=0.3, random_rotation=0.3)

In [25]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='Input_layer')
x = aug_layer(inputs)
x = base_model(x, training=False)
x = GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(No_of_classes, activation='softmax', dtype=tf.float32)(x)

model_1 = Model(inputs, outputs, name='Mobilenetv3lnew')

model_1.summary()

Model: "Mobilenetv3lnew"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 1280)              5120      
 chNormalization)                                  

In [26]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [27]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [28]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [29]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 20, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/20


2024-07-26 23:41:40.853393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-26 23:41:40.935542: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-26 23:41:42.028228: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563cf216b4a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-26 23:41:42.028243: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-26 23:41:42.038326: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 23:41:42.119472: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 12.6077 - accuracy: 0.5693
Epoch 1: val_loss improved from inf to 10.28566, saving model to Trained_Models/Mobilenetv3lnew
74/74 [==============================] - 36s 431ms/step - loss: 12.6077 - accuracy: 0.5693 - val_loss: 10.2857 - val_accuracy: 0.8497 - lr: 0.0010
Epoch 2/20
74/74 [==============================] - ETA: 0s - loss: 8.2778 - accuracy: 0.6874
Epoch 2: val_loss improved from 10.28566 to 6.54520, saving model to Trained_Models/Mobilenetv3lnew
74/74 [==============================] - 29s 385ms/step - loss: 8.2778 - accuracy: 0.6874 - val_loss: 6.5452 - val_accuracy: 0.8863 - lr: 0.0010
Epoch 3/20
74/74 [==============================] - ETA: 0s - loss: 5.2534 - accuracy: 0.7366
Epoch 3: val_loss improved from 6.54520 to 4.39690, saving model to Trained_Models/Mobilenetv3lnew
74/74 [==============================] - 27s 366ms/step - loss: 5.2534 - accuracy: 0.7366 - val_loss: 4.3969 - val_accuracy: 0.7122 - lr: 0.0

In [34]:
model_1.evaluate(test_datagen)
base_model.trainable = True
for layer in base_model.layers[:-5]:
    layer.trainable = False

27/27 [==============================] - 1s 25ms/step - loss: 30.8310 - accuracy: 0.2462


In [35]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True


In [36]:
start_epoch = 20
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Mobilenetv3lnew"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 1280)              5120      
 chNormalization)                                  

In [ ]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10,initial_epoch=start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 0.8288 - accuracy: 0.7048
Epoch 21: val_loss did not improve from 0.77880
74/74 [==============================] - 16s 173ms/step - loss: 0.8288 - accuracy: 0.7048 - val_loss: 132.9536 - val_accuracy: 0.0000e+00 - lr: 3.0000e-04
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.7970 - accuracy: 0.7048
Epoch 22: val_loss did not improve from 0.77880
74/74 [==============================] - 12s 160ms/step - loss: 0.7970 - accuracy: 0.7048 - val_loss: 103.5913 - val_accuracy: 0.0000e+00 - lr: 3.0000e-04
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.8362 - accuracy: 0.6997
Epoch 23: val_loss did not improve from 0.77880
74/74 [==============================] - 12s 157ms/step - loss: 0.8362 - accuracy: 0.6997 - val_loss: 184.3490 - val_accuracy: 0.0000e+00 - lr: 3.0000e-04
Epoch 24/30
17/74 [=====>........................] - ETA: 8s - loss: 0.9017 - accuracy: 0.6581

In [30]:
MobileNet_Best = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Mobilenetv2")


In [31]:
MobileNet_Best.evaluate(test_datagen)

27/27 [==============================] - 3s 104ms/step - loss: 0.2928 - accuracy: 0.8828


[0.2927977740764618, 0.8828402161598206]